In [1]:
# import libraries 
import mineapy
from cobra.io import load_matlab_model,load_json_model
import pandas as pd
from mineapy.core.taskEnrich import TaskEnrichment
from mineapy.core.thermo_model import ThermoModel_WithoutInfo
from mineapy.core.rxnExp import ReactionExp
import sys
import os
import pickle

# Add the path to the tutorials directory to sys.path
sys.path.append(os.path.abspath(os.path.join('..')))

#print(os.path.abspath(os.path.join('..')))
from data_utility import getErythromycin_data

Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-20


## Data Analysis Using MiNEApy (Genome-Scale Model)

This tutorial demonstrates the application of Minimum Network Enrichment Analysis (MiNEA) using the MiNEApy package, applied to the same transcriptomic data used in Tutorial 1 (core metabolism model) in response to erythromycin treatment. The analysis remains the same but is now conducted on a more comprehensive genome-scale model. The two key parts of the analysis are:

1. [**Effect of Erythromycin on Gene Regulation:**](#effect-of-erythromycin-on-gene-regulation)
The first analysis examines which minimal networks are upregulated or downregulated in response to erythromycin treatment. Gene expression levels from erythromycin-treated cells are compared with control cells treated with ethanol. This comparison identifies the metabolic networks most impacted by erythromycin.
   
2. [**FPKM Values and Network Enrichment:**](#fpkm-values-and-network-enrichment)  
FPKM values from the erythromycin-treated group are used to identify networks enriched by highly expressed genes, minimizing the influence of low-expression genes, as done in the core metabolism model.



<a name="effect-of-erythromycin-on-gene-regulation"></a>
### 1. Effect of Erythromycin on Gene Regulation
As done in Tutorial 1, load the gene expression data, map genes to reactions using GPR rules, compute minimal networks with MiNEApy, and perform enrichment analysis to identify impacted pathways.

In [2]:
## load the gene expression data for gene expression   Erythromycin 
ery_df=getErythromycin_data()

ery_df['significant(ERYvsETH)'].unique()
# Convert the 'log2FoldChange(ERYvsETH)' column to numeric, coercing errors to NaN
ery_df['log2FoldChange(ERYvsETH)'] = pd.to_numeric(ery_df['log2FoldChange(ERYvsETH)'], errors='coerce')

## find up and down regulated genes 
up_down_df=ery_df[(ery_df['significant(ERYvsETH)']=='UP')|(ery_df['significant(ERYvsETH)']=='DOWN')]
up_down_df['fold_change']=2 ** up_down_df['log2FoldChange(ERYvsETH)']

print(up_down_df)


     Gene_id  ERY_group_fpkm  log2FoldChange(ERYvsETH)  pval(ERYvsETH)  \
3      b0004      226.403292                  -0.48068    8.902500e-03   
4      b0005       43.621743                  -1.36250    1.592700e-03   
6      b0007        9.651617                  -0.96696    1.022300e-04   
7      b0008     3325.924410                   1.17350    7.830000e-11   
12     b0014     1091.185448                  -1.78210    1.500000e-12   
...      ...             ...                       ...             ...   
4477   b4691       40.244913                  -2.28000    7.070000e-17   
4479   b4693        1.282623                  -1.85350    2.843400e-04   
4482   b4696       11.717341                  -1.32330    2.900000e-11   
4488   b4702      181.287835                   2.32600    2.270000e-08   
4490   b4704      674.641886                  -2.47200    6.040000e-05   

      padj(ERYvsETH) significant(ERYvsETH)  fold_change  
3       2.142000e-02                  DOWN     0.7166

/Users/vpandey/opt/miniconda3/envs/test37/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [3]:
# load core metabolic genome_scale_model
cobra_model = load_json_model('../models/iJO1366.json')
genes=[g.id for g in cobra_model.genes]
sol=cobra_model.optimize()
print(sol,type(cobra_model.solver))


<Solution 0.814 at 0x7fe29a57c750> <class 'optlang.gurobi_interface.Model'>


In [4]:
### working with condition comparisons
gene_reg={'gene_id':up_down_df['Gene_id'].to_list(),'fold_change':up_down_df['fold_change'].to_list(),'up_cutoff':1.1,'down_cutoff':0.9}

reg_analysis=ReactionExp(cobra_model,gene_reg=gene_reg)
reg_analysis
params_rxns={'up_rxns':reg_analysis.up_rxns,'down_rxns':reg_analysis.down_rxns}
print(params_rxns)

2024-12-10 09:42:07,307 - expression_logger - INFO - start analysis of comparison between conditions .....


 gene value is not found or nan for the reaction = 12PPDRtex
 gene value is not found or nan for the reaction = 12PPDRtex
 gene value is not found or nan for the reaction = 12PPDStex
 gene value is not found or nan for the reaction = 12PPDStex
 gene value is not found or nan for the reaction = 23CAMPtex
 gene value is not found or nan for the reaction = 23CAMPtex
 gene value is not found or nan for the reaction = 23CCMPtex
 gene value is not found or nan for the reaction = 23CCMPtex
 gene value is not found or nan for the reaction = 23CGMPtex
 gene value is not found or nan for the reaction = 23CGMPtex
 gene value is not found or nan for the reaction = 23CUMPtex
 gene value is not found or nan for the reaction = 23CUMPtex
 gene value is not found or nan for the reaction = 23DAPPAtex
 gene value is not found or nan for the reaction = 23DAPPAtex
 gene value is not found or nan for the reaction = 23PDE2pp
why or_vals are empty
 gene value is not found or nan for the reaction = 23PDE4pp
wh

### Configuring the `parameter.yaml` for MiNEA

In this section, we will walk through how to configure the `parameter.yaml` file for running MiNEA (Minimum Network Enrichment Analysis). This file controls the behavior of the analysis and allows you to tailor the process according to your specific research needs. Below is an example `parameter.yaml` configuration with explanations for each section:

```yaml
# Biomass reactions: Specify the biomass reaction(s) used in your model.
# Uncomment the lines below to define specific biomass reactions.
biomass_rxns:
    #- Biomass_Ecoli_core
    #- Ec_biomass_iJO1366_WT_53p95M
    - BIOMASS_Ecoli_core_w_GAM

# Define the minimal growth rate required. Setting to 'auto' uses 95% 
# of the maximum growth rate determined by TFA or FBA.
growth_rate: auto

# Define the Metabolic Tasks (MTs). These are specific metabolites 
# for which you want to find the minimal networks.
MTs: ['pyr_c', 'oaa_c','succ_c']

# Use Building Blocks (BBB) as Metabolic Tasks if set to 'yes'.
MT_BBB: yes

# Define the medium composition. Specify which external metabolites 
# are available in the medium. Reactions not listed will retain 
# their model-defined bounds.
medium:
    #EX_glc__D_e: -10

# Cofactor pairs: Specify pairs of cofactors that are to be balanced 
# during the analysis.
cofactor_pairs:
    - ['atp_c', 'adp_c']
    - ['nad_c', 'nadh_c']
    - ['nadp_c', 'nadph_c']
    - ['accoa_c', 'coa_c']

# Lump method: This parameter will alllow maximum number of network size. 
# Options include 'Min', 'Min+1', 'Min+30'.
lump_method: Min+30

# Maximum number of lumped reactions per metabolite or metabolic task. Only applicable 
# if 'Min' or 'Min+p' is selected as lump method. 
# It defines the number of alternative networks that can be obtained for each metabolite.
max_lumps_per_BBB: 2

# Diverge minimum: Specifies how much the solutions are allowed to deviate from the base solution. 
# A higher value permits more variation between solutions, indicating the number of reactions that can differ between consecutive solutions. publication: Rezola A, Pey J, de Figueiredo LF, Podhorski A, Schuster S, Rubio A, Planes FJ. Selection of human tissue-specific elementary flux modes using gene expression data. Bioinformatics. 2013 Aug 15;29(16):2009-16. doi: 10.1093/bioinformatics/btt328. Epub 2013 Jun 6. PMID: 23742984.
diverge_min: 5  # default=1 

# Solver options: Define which solver to use. 'auto' lets the program decide. 'gurobi', 'cplex','glpk' are the options. 
solver: auto

# Solver options: Define which solver to use. 'auto' lets the program decide.
solver: auto

# Force solving even if some tasks are infeasible.
force_solve: False

# Timeout: Maximum time (in minutes) allocated for solving each problem.
timeout: 300

# Feasibility tolerance: Define the precision for feasibility checks.
feasibility: 1e-9

# Constraint method: Choose how to handle constraints. Options 
# are 'flux', 'integer', or 'both'.
constraint_method: both

sum_flux: True 

sum_flux: True  <span style="color:red">**IMPORTANT:** This minimizes the sum of fluxes through reactions, which speeds up computation. However, it does not guarantee that the number of reactions is minimized. If set to False, it minimizes the number of reactions, but this can significantly slow down the process for some metabolic tasks and genome-scale models.</span>


In [5]:
# here one can modify parameters of MiNEA 
path_to_params = '../input/task_enrichment_gem_params.yaml'

In [6]:
# apply MiNEA with up and down regulated reactions
task_enrich = TaskEnrichment(cobra_model,path_to_params,params_rxns)

task_enrich.run()


Opened parameters file
Read LP format model from file /var/folders/ty/spqw6k0n4dn1yvpgn_swvjm80000gn/T/tmp1woyo4g3.lp
Reading time = 0.01 seconds
: 1807 rows, 5170 columns, 20334 nonzeros
Read LP format model from file /var/folders/ty/spqw6k0n4dn1yvpgn_swvjm80000gn/T/tmpseqti32k.lp
Reading time = 0.01 seconds
: 1807 rows, 5170 columns, 20334 nonzeros
Read LP format model from file /var/folders/ty/spqw6k0n4dn1yvpgn_swvjm80000gn/T/tmpuqxp36qt.lp
Reading time = 0.01 seconds
: 1807 rows, 5170 columns, 20334 nonzeros


2024-12-10 09:42:24,694 - ME modelNone - INFO - # Model initialized 
2024-12-10 09:42:24,695 - ME modelNone - INFO - # Model conversion starting...
2024-12-10 09:42:33,850 - ME modelNone - INFO - # Model conversion done.
2024-12-10 09:42:33,851 - ME modelNone - INFO - # Updating cobra_model variables...
2024-12-10 09:42:33,873 - ME modelNone - INFO - # cobra_model variables are up-to-date
2024-12-10 09:42:33,874 - ME modelNone - INFO - Setting minimal growth rate to 95% of the TFA solution
2024-12-10 09:42:34,193 - ME modelNone - INFO - Setting minimal growth rate to 0.813999106676092
2024-12-10 09:42:34,193 - ME modelNone - INFO - Enumerating minmal networks ...


Timeout limit is 1000s


/Users/vpandey/opt/miniconda3/envs/test37/lib/python3.7/site-packages/sympy/__init__.py:676: SymPyDeprecationWarning: 

importing sympy.core.numbers with 'from sympy import *' has been
deprecated since SymPy 1.6. Use import sympy.core.numbers instead. See
https://github.com/sympy/sympy/issues/18245 for more info.

  deprecated_since_version="1.6").warn()
2024-12-10 09:42:43,233 - ME modelNone - INFO - # Model conversion starting...


Preparing metabolic tasks...


2024-12-10 09:42:46,611 - ME modelNone - INFO - # Model conversion done.
2024-12-10 09:42:46,612 - ME modelNone - INFO - # Updating cobra_model variables...
2024-12-10 09:42:46,662 - ME modelNone - INFO - # cobra_model variables are up-to-date
met=pyr_c:   0%|          | 0/1 [00:00<?, ?it/s]

Min network method detected: min+30
Produced 15.096000000000002 with 218 reactions deactivated
Produced 15.096000000000002 with 218 reactions deactivated
Produced 15.096000000000002 with 218 reactions deactivated
Produced 15.096000000000002 with 218 reactions deactivated
Produced 15.096000000000002 with 218 reactions deactivated


met=pyr_c: 100%|██████████| 1/1 [00:04<00:00,  4.73s/it]
2024-12-10 09:42:51,397 - ME modelNone - INFO - Enumeration is done. now save the result in pickle file
2024-12-10 09:42:51,399 - ME modelNone - INFO - Enumerating minimal networks ...


As described in tutorial1_ecoli_core, MiNEA generates the following key output files:

1. **fva.pickle:** Contains Flux Variability Analysis (FVA) results, showing the range of possible fluxes for each reaction under given constraints.

2. **mins.pickle:** Stores minimal networks for each task, representing the smallest reaction sets capable of performing specific metabolic tasks.

3. **upregulated_rxns.txt:** Provides minimal networks ranked by p-values for upregulated reactions, emphasizing upregulated reactions while minimizing downregulated ones.

4. **downregulated_rxns.txt:** Provides minimal networks ranked by p-values for downregulated reactions, focusing on downregulated reactions while minimizing upregulated ones.

5. **deregulated_rxns.txt:** Combines both upregulated and downregulated reactions, offering a comprehensive view of deregulation in the metabolic network.

<a name="fpkm-values-and-network-enrichment"></a>
### 2. FPKM Values and Network Enrichment
To achieve this analysis step by step:
- Load the gene expression data to identify the high- and low-expressed gene genes.
- Using Gene-Protein-Reaction (GPR) rules, compute high- and low-expressed reactions based on the gene expression.
- Compute minimal networks by tuning parameters in the MiNEApy tool. 
- Combine high- and low-expressed reactions  and minimal networks to perform enrichment analysis and discover which pathways or networks are significantly enriched by high-expressed reactions.
- publication:
1. Rezola A, Pey J, de Figueiredo LF, Podhorski A, Schuster S, Rubio A, Planes FJ. Selection of human tissue-specific elementary flux modes using gene expression data. Bioinformatics. 2013 Aug 15;29(16):2009-16. doi: 10.1093/bioinformatics/btt328. Epub 2013 Jun 6. PMID: 23742984.
2. Pandey V, Hatzimanikatis V. Investigating the deregulation of metabolic tasks via Minimum Network Enrichment Analysis (MiNEA) as applied to nonalcoholic fatty liver disease using mouse and human omics data. PLoS Comput Biol. 2019 Apr 19;15(4):e1006760. doi: 10.1371/journal.pcbi.1006760. PMID: 31002661; PMCID: PMC6493771. 

In [7]:
# run MiNEA for low and high expression reactions 
gene_exp={'gene_id':ery_df['Gene_id'].to_list(),'exp_val':ery_df['ERY_group_fpkm'].to_list(),'high_cutoff':0.15,'low_cutoff':0.15}

exp_analysis=ReactionExp(cobra_model,gene_exp=gene_exp)

params_rxns={'high_rxns':exp_analysis.high_rxns,'low_rxns':exp_analysis.low_rxns}

task_enrich = TaskEnrichment(cobra_model,path_to_params,params_rxns)

task_enrich.run()

2024-12-10 09:42:51,615 - expression_logger - INFO - start analysis of context (tissue-specific).....


 gene value is not found or nan for the reaction = ACALDtpp
why or_vals are empty
 gene value is not found or nan for the reaction = ACONIs
why or_vals are empty
 gene value is not found or nan for the reaction = AOBUTDs
why or_vals are empty
 gene value is not found or nan for the reaction = ARBTNexs
why or_vals are empty
 gene value is not found or nan for the reaction = ATPHs
why or_vals are empty
 gene value is not found or nan for the reaction = CBMD
why or_vals are empty
 gene value is not found or nan for the reaction = CO2tpp
why or_vals are empty
 gene value is not found or nan for the reaction = CPGNexs
why or_vals are empty
 gene value is not found or nan for the reaction = DATPHs
why or_vals are empty
 gene value is not found or nan for the reaction = DHPTDCs2
why or_vals are empty
 gene value is not found or nan for the reaction = DMSOtpp
why or_vals are empty
 gene value is not found or nan for the reaction = ETOHtrpp
why or_vals are empty
 gene value is not found or nan 

2024-12-10 09:43:08,925 - ME modelNone - INFO - # Model initialized 
2024-12-10 09:43:08,926 - ME modelNone - INFO - # Model conversion starting...
2024-12-10 09:43:18,923 - ME modelNone - INFO - # Model conversion done.
2024-12-10 09:43:18,924 - ME modelNone - INFO - # Updating cobra_model variables...
2024-12-10 09:43:18,952 - ME modelNone - INFO - # cobra_model variables are up-to-date
2024-12-10 09:43:18,953 - ME modelNone - INFO - Setting minimal growth rate to 95% of the TFA solution
2024-12-10 09:43:19,258 - ME modelNone - INFO - Setting minimal growth rate to 0.813999106676092
2024-12-10 09:43:19,258 - ME modelNone - INFO - Enumerating minmal networks ...


Timeout limit is 1000s


2024-12-10 09:43:28,730 - ME modelNone - INFO - # Model conversion starting...


Preparing metabolic tasks...


2024-12-10 09:43:31,808 - ME modelNone - INFO - # Model conversion done.
2024-12-10 09:43:31,809 - ME modelNone - INFO - # Updating cobra_model variables...
2024-12-10 09:43:31,844 - ME modelNone - INFO - # cobra_model variables are up-to-date
met=pyr_c:   0%|          | 0/1 [00:00<?, ?it/s]

Min network method detected: min+30
Produced 15.096000000000002 with 218 reactions deactivated
Produced 15.096000000000002 with 218 reactions deactivated
Produced 15.096000000000002 with 218 reactions deactivated
Produced 15.096000000000002 with 218 reactions deactivated
Produced 15.096000000000002 with 218 reactions deactivated


met=pyr_c: 100%|██████████| 1/1 [00:04<00:00,  4.85s/it]
2024-12-10 09:43:36,700 - ME modelNone - INFO - Enumeration is done. now save the result in pickle file
2024-12-10 09:43:36,703 - ME modelNone - INFO - Enumerating minimal networks ...


### Output Files Generated by MiNEA with High and Low Expressed Reactions

In addition to the output files previously described 
(fva.pickle and mins.pickle), which store the results of 
Flux Variability Analysis (FVA) and enumerated minimal networks 
respectively, MiNEA generates a third file focused on 
gene expression levels:

3. **high_expressed_rxns.txt**:

   This file contains the enrichment result tables 
   for highly expressed reactions. Each alternative 
   minimal network is ranked based on p-values. The 
   computation of p-values in this context prioritizes 
   the inclusion of as many high expressed reactions 
   as possible while minimizing the inclusion of low 
   expressed reactions. This type of network analysis 
   is particularly useful when identifying tissue-specific 
   metabolic networks or when trying to understand the 
   metabolic context of specific biological conditions.
